In [1]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
saveDir <- "environment/brain_sim_nodtu_lowfc/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees/DE"
load(file=file.path(saveDir, "tseConsUp.RData"))
suppressPackageStartupMessages(source("tree_helper_function.R"))

In [9]:
load(file.path(saveDir, "tAfterBHMoreClimbMIRVUp.RData"))
load(file.path(saveDir, "bouthBrain.RData"))
load(file.path(saveDir, "bSwishCons.RData"))

In [5]:
which(bouthBrain[[1]][["results.by.node"]][["is.driver"]])
which(bouthBrain[[2]][["results.by.node"]][["is.driver"]])
which(bouthBrain[[3]][["results.by.node"]][["is.driver"]])

[1] 1

[1] 1

[1] 1

In [6]:
load("environment/brain_sim_nodtu_lowfc/simulate.rda")
suppressPackageStartupMessages(library(TreeSummarizedExperiment))
suppressPackageStartupMessages(library(ape))
tree <- rowTree(tseConsUp)
l <- length(tree$tip)
missingTxps <- setdiff(tree$tip.label, rownames(sim.counts.mat))
dts <- Reduce(union, c(names(iso.dge)[iso.dge], names(iso.dte)[iso.dte], names(iso.dte.only)[iso.dte.only], 
                       names(iso.dtu)[iso.dtu]))
dts <- intersect(dts, rownames(sim.counts.mat))
sim.counts.mat <- rbind(sim.counts.mat, matrix(0, nrow = length(missingTxps), ncol = ncol(sim.counts.mat),
                                               dimnames = list(missingTxps, colnames(sim.counts.mat))))
sim.counts.mat <- sim.counts.mat[tree$tip,]
dts <- intersect(dts, rownames(sim.counts.mat))

innNodes <- l+1:tree$Nnode
aggCountsNodes <- computeAggNodesU(tree, c(1:length(tree$tip),innNodes), sim.counts.mat)
logFCNodes <- ifelse(rowSums(aggCountsNodes)==0, 0, log2(aggCountsNodes[,2]+1)-log2(aggCountsNodes[,1]+1))
names(logFCNodes)[(l+1):length(logFCNodes)] <- paste("Node", names(logFCNodes)[(l+1):length(logFCNodes)], sep="")
rootFC <- abs(logFCNodes[l+1])

In [10]:
y <- tseConsUp[tree$tip.label,]
y <- fishpond::scaleInfReps(y)
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")
detNodes <- list()
detNodes[["Txps"]] <- lapply(c(0.01, 0.05, 0.1), function(x) which(mcols(y)[,"qvalue"] <= x ))
detNodes[["tAfterBHMoreClimbMIRV=0.4"]] <- lapply(tAfterBHMoreClimbMIRVUp[["mIRV=0.4"]], function(ta) ta)
detNodes[["treeClimbR"]] <- lapply(bSwish, function(sw) sw$output[sw$output$signal.node,][["node"]])
detNodes[["treeClimbRO"]] <- lapply(bSwish, function(sw) unlist(Descendants(tree,sw$output[sw$output$signal.node,][["node"]])))
                                                  

negNodes <- list()
negNodes[["Txps"]] <- lapply(detNodes[["Txps"]], function(nodes) setdiff(seq(l), nodes))
negNodes[["tAfterBHMoreClimbMIRV=0.4"]] <- lapply(detNodes[["tAfterBHMoreClimbMIRV=0.4"]], function(nodes) setdiff(seq(nrow(y)), unlist(Descendants(tree, nodes))))
detNodes[["treeClimbR"]] <- lapply(bSwish, function(sw) sw$output[sw$output$signal.node,][["node"]])
detNodes[["treeClimbRO"]] <- lapply(bSwish, function(sw) unlist(Descendants(tree,sw$output[sw$output$signal.node,][["node"]])))

In [13]:
sapply(detNodes[["treeClimbRO"]],length)

[1] 1801 3274 4801

In [11]:
df <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(detNodes))
{    
    print(n)
    stats <-  sapply(seq_along(detNodes[[n]]), function(i) computeMetOut(detNodes[[n]][[i]], logFCNodes, tree = tree, negNodes = negNodes[[n]][[i]], lfcThresh = rootFC))
    colnames(stats) <- colnames(df)[c(3:5)]
    stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
    df <- rbind(df, stats)
}
suppressPackageStartupMessages(library(tidyr))
dfA <- df %>% pivot_wider(names_from = Metric, values_from = c(3:5))
dfA <- dfA[,c(1,seq(2,7,2),seq(3,7,2))]
dfA

[1] "Txps"
[1] "tp 0.99184339314845"
[1] "tp 0.960952380952381"
[1] "tp 0.919753086419753"
[1] "tAfterBHMoreClimbMIRV=0.4"
[1] "tp 0.992207792207792"
[1] "tp 0.954972465176547"
[1] "tp 0.911352209579254"
[1] "treeClimbR"
[1] "tp 0"


ERROR: Error in `[.default`(tab, 2, 2): subscript out of bounds


In [23]:
detInnerNodes <- sapply(detNodes[["tAfterBHMoreClimbMIRV=0.4"]], function(nodes) nodes[nodes > l])

In [28]:
sapply(detInnerNodes, length)

[1]  568  911 1020

In [26]:
sapply(seq_along(detInnerNodes), function(i) mean(abs(logFCNodes[detInnerNodes[[i]]]) < rootFC))

[1] 0.001760563 0.037321625 0.089215686

In [38]:
missP <- setdiff(unlist(Descendants(tree,detNodes[[3]][[2]])), detNodes[[1]][[2]])
missTP <- missP[abs(logFCNodes[missP]) > rootFC] ##True positive transcripts that have an internal node

missInTreeDE <- setdiff(detNodes[[1]][[2]], unlist(Descendants(tree,detNodes[[3]][[2]])))
missTPInTreeDE <- missInTreeDE[abs(logFCNodes[missInTreeDE]) > rootFC] ##True positive transcripts that have an internal node

In [39]:
length(missP)
length(missTP)
length(missInTreeDE)
length(missTPInTreeDE)

[1] 1984

[1] 1136

[1] 285

[1] 238

In [35]:
sum(unique(unlist(Ancestors(tree, missTP))) %in% detNodes[[3]][[2]]) ### Nodes of these additional ancestors 

[1] 438

In [37]:
names(detNodes)

[1] "Txps"                      "tAfterBHMoreClimbMIRV=0"  
[3] "tAfterBHMoreClimbMIRV=0.4" "tAfterBHMoreClimbMIRV=1"  
[5] "tAfter"                    "tAfterNoSign"             
[7] "tAfterNoStop"

In [34]:
length(missP)
length(missTP)
length(missInTreeDE)
length(missTPInTreeDE)



[1] 1984

[1] 1136

[1] 285

[1] 238

#### Mean

In [14]:
load(file=file.path(saveDir, "tseMean.RData"))
load("environment/brain_sim_nodtu_lowfc/simulate.rda")
suppressPackageStartupMessages(library(TreeSummarizedExperiment))
suppressPackageStartupMessages(library(ape))
tree <- rowTree(tse)
l <- length(tree$tip)
missingTxps <- setdiff(rownames(tse)[1:l], rownames(sim.counts.mat))
dts <- Reduce(union, c(names(iso.dge)[iso.dge], names(iso.dte)[iso.dte], names(iso.dte.only)[iso.dte.only], 
                       names(iso.dtu)[iso.dtu]))
dts <- intersect(dts, rownames(sim.counts.mat))
sim.counts.mat <- rbind(sim.counts.mat, matrix(0, nrow = length(missingTxps), ncol = ncol(sim.counts.mat),
                                               dimnames = list(missingTxps, colnames(sim.counts.mat))))
sim.counts.mat <- sim.counts.mat[rownames(tse)[1:l],]
dts <- intersect(dts, rownames(sim.counts.mat))

innNodes <- l+1:tree$Nnode
aggCountsNodes <- computeAggNodesU(tree, c(1:length(tree$tip),innNodes), sim.counts.mat)
logFCNodes <- ifelse(rowSums(aggCountsNodes)==0, 0, log2(aggCountsNodes[,2]+1)-log2(aggCountsNodes[,1]+1))
names(logFCNodes)[(l+1):length(logFCNodes)] <- paste("Node", names(logFCNodes)[(l+1):length(logFCNodes)], sep="")
rootFC <- abs(logFCNodes[l+1])

In [15]:
load(file.path(saveDir, "tAfterBHMoreClimbMIRVMean.RData"))
y <- tse[1:l,]
y <- fishpond::scaleInfReps(y)
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")

In [16]:
detNodes <- list()
detNodes[["Txps"]] <- lapply(c(0.01, 0.05, 0.1), function(x) which(mcols(y)[,"qvalue"] <= x ))
detNodes[["tAfterBHMoreClimbMIRV=0"]] <- lapply(tAfterBHMoreClimbMIRV[[1]], function(ta) ta)
detNodes[["tAfterBHMoreClimbMIRV=0.4"]] <- lapply(tAfterBHMoreClimbMIRV[[2]], function(ta) ta)
detNodes[["tAfterBHMoreClimbMIRV=1"]] <- lapply(tAfterBHMoreClimbMIRV[[3]], function(ta) ta)

negNodes <- list()
negNodes[["Txps"]] <- lapply(detNodes[["Txps"]], function(nodes) setdiff(seq(l), nodes))
negNodes[["tAfterBHMoreClimbMIRV=0"]] <- lapply(detNodes[["tAfterBHMoreClimbMIRV=0"]], function(nodes) setdiff(seq(nrow(y)), unlist(Descendants(tree, nodes))))
negNodes[["tAfterBHMoreClimbMIRV=0.4"]] <- lapply(detNodes[["tAfterBHMoreClimbMIRV=0.4"]], function(nodes) setdiff(seq(nrow(y)), unlist(Descendants(tree, nodes)))) 
negNodes[["tAfterBHMoreClimbMIRV=1"]] <- lapply(detNodes[["tAfterBHMoreClimbMIRV=1"]], function(nodes) setdiff(seq(nrow(y)), unlist(Descendants(tree, nodes)))) 


In [17]:
df <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(detNodes))
{    
    print(n)
    stats <-  sapply(seq_along(detNodes[[n]]), function(i) computeMetOut(detNodes[[n]][[i]], logFCNodes, tree = tree, negNodes = negNodes[[n]][[i]], lfcThresh = rootFC))
    colnames(stats) <- colnames(df)[c(3:5)]
    stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
    df <- rbind(df, stats)
}
dfA <- df %>% pivot_wider(names_from = Metric, values_from = c(3:5))
dfA <- dfA[,c(1,seq(2,7,2),seq(3,7,2))]
dfA                     

[1] "Txps"
[1] "tp 0.991852254209669"
[1] "tp 0.960747071858183"
[1] "tp 0.92046332046332"
[1] "tAfterBHMoreClimbMIRV=0"
[1] "tp 0.998370008149959"
[1] "tp 0.950342465753425"
[1] "tp 0.897489539748954"
[1] "tAfterBHMoreClimbMIRV=0.4"
[1] "tp 0.999211356466877"
[1] "tp 0.964801126363956"
[1] "tp 0.925688839409964"
[1] "tAfterBHMoreClimbMIRV=1"
[1] "tp 0.999174236168456"
[1] "tp 0.968443197755961"
[1] "tp 0.931119648737651"


Method,FDR_0.01_FDR,FDR_0.05_FDR,FDR_0.10_FDR,FDR_0.01_TPR,FDR_0.05_TPR,FDR_0.10_TPR
<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>
Txps,0.008147746,0.03925293,0.07953668,0.2080201,0.3457507,0.4073821
tAfterBHMoreClimbMIRV=0,0.001629992,0.04965753,0.1025105,0.2019786,0.4026115,0.4796869
tAfterBHMoreClimbMIRV=0.4,0.0007886435,0.03519887,0.07431116,0.1634839,0.3613235,0.4369991
tAfterBHMoreClimbMIRV=1,0.0008257638,0.0315568,0.06888035,0.1471661,0.3424675,0.419303
